<a href="https://colab.research.google.com/github/Galera115/PDIH_UGR/blob/main/TRABAJO%20TEOR%C3%8DA/PoseGetVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install opencv-python
!pip install mediapipe

from google.colab import files

uploaded = files.upload()

--2022-05-01 17:58:32--  https://i.gifer.com/embedded/download/8O30.gif
Resolving i.gifer.com (i.gifer.com)... 54.39.157.175
Connecting to i.gifer.com (i.gifer.com)|54.39.157.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7231638 (6.9M) [text/plain]
Saving to: ‘8O30.gif’

8O30.gif            100%[===================>]   6.90M  12.2MB/s    in 0.6s    

2022-05-01 17:58:33 (12.2 MB/s) - ‘8O30.gif’ saved [7231638/7231638]



In [44]:
import cv2
import mediapipe as mp
import time
from google.colab.patches import cv2_imshow


class PoseDetector:

    def __init__(self, mode = False, upBody = False, smooth=True, detectionCon = 0.5, trackCon = 0.75):

        self.mode = mode
        self.upBody = upBody
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.upBody, self.smooth, False ,self.detectionCon, self.trackCon)

    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        #print(results.pose_landmarks)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS)

        return img

    def getPosition(self, img, draw=True):
        lmList= []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                #print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return lmList


cap = cv2.VideoCapture('/content/a.mp4')
pTime = 0
img_array = []
detector = PoseDetector()
success, img = cap.read()
while img is not None:
    img = detector.findPose(img)
    lmList = detector.getPosition(img)
    img_array.append(img)
    #print(lmList)

    cTime = time.time()
    fps = 1 / (cTime - pTime)

    pTime = cTime
    #cv2.putText(img, str(int(fps)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    #cv2_imshow(img)
    #cv2.waitKey(1)
    success, img = cap.read()


In [47]:
height, width, layers = img_array[0].shape
size = (width,height)
out = cv2.VideoWriter('1.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 20.0, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()